# Portfolio Optimization
This notebook demonstrates portfolio optimization techniques on portfolios of N assets. Bootstrapping will be used to estimate the standard errors of the allocation optimization functions. A simple example of two assets is first used to conceptualize the optimization techniques. Once demonstrated, portfolio optimization will be scalled to N assets.

In [72]:
# Notebook imports
import numpy as np
import pandas as pd
import yfinance as yf

### 1. Minimum Variance
Assets yield returns. Suppose we are a risk-averse and would like to allocate our money between two assets with returns X and Y, where X and Y are random continuous variables. We want to determine the allocation that minimizes total portfolio risk (variance). $\alpha$ denotes the allocation to X and $1-\alpha$ denotes the allocation to Y. The total portfolio is then represented as the equation $\alpha*X + (1-\alpha)*Y$. The variance of the portfolio is:<br><br>
$Var(\alpha*X + (1-\alpha)*Y)$<br><br>
This is transformed into a minimization problem.

In [73]:
X = 'SPY' # S&P 500 Index
Y = 'DIA' # Dow-Jones Industrial Average

# Download monthly data
data = yf.download(tickers=[X, Y], interval='1mo', period='max', auto_adjust=True)['Close']

# Calculate the monthly returns
data[X] = data[X].pct_change()
data[Y] = data[Y].pct_change()

# Clean the data
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop(columns='Date', inplace=True)

# Display the summary statistics of the returns
print(data.describe())

[*********************100%***********************]  2 of 2 completed

Ticker         DIA         SPY
count   328.000000  328.000000
mean      0.007743    0.007796
std       0.043895    0.045029
min      -0.154254   -0.160354
25%      -0.016279   -0.016705
50%       0.010642    0.013423
75%       0.031763    0.034857
max       0.142675    0.133610


In [74]:
def alpha_function(df, idx, X='SPY', Y='DIA'):
    """
    * alpha_function()
    * Calculates the alpha parameter, allocation to the X asset in a portfolio
    * minimizing variance between X and Y.
    *
    * df:  Dataframe of asset returns
    * idx: Index defining observations to use in the calculations
    * X:   Label of the X asset
    * Y:   Label of the Y asset
    *
    * returns the alpha parameter
    """

    covariance = np.cov(df[[X, Y]].loc[idx], rowvar=False)

    # Minimized portfolio variance
    return (covariance[1,1] - covariance[0,1]) / (covariance[0,0] + covariance[1,1] - 2 * covariance[0,1])

alpha = alpha_function(data, range(len(data)))
print(f'{X} weight = {alpha:.4f} | {Y} weight = {(1-alpha):.4f}')

SPY weight = 0.2812 | DIA weight = 0.7188


In [75]:
def bootstrapped_alpha_SE(alpha_func, data, B=1000, seed=0):
    """
    * bootstrap_alpha_SE()
    * Calculates the estimated standard error for the alpha parameter using the bootstrap
    * technique. Makes no assumption as to the alpha function (i.e. can be for 2-n assets).
    *
    * alpha_func: Function to calculate the alpha parameter
    * data: Dataframe of asset returns
    * B:    Number of bootstrap replications
    * seed: Random seed for reproducability
    """

    random_rng = np.random.default_rng(seed)
    n = len(data) # Number of observations to use in the index range

    # Standard error parameters
    se_first, se_second = 0, 0

    for _ in range(B):
        # Index range with replacement
        idx = random_rng.choice(data.index, n, replace=True)

        alpha_value = alpha_function(data, idx)

        # Update the standard error parameters
        se_first += alpha_value
        se_second += alpha_value ** 2

    return np.sqrt(se_second / B - (se_first / B) ** 2)

# Compute the estimated alpha standard error
alpha_se = bootstrapped_alpha_SE(alpha_function, data)

print(alpha_se)

0.20038974041602853
